Routine, die das Bild einliest (Befehl: cv2.imread()) und die
einzelnen Grauwertstufen als Unterbilder aus dem Originalbild ausliest.
Bestimmung des Mittelwertes und der Standardabweichung für jede Stufe und die der Grau
werte.

In [2]:
import numpy as np
import os
import cv2

# get current working directory
path = os.getenv('DATA_PATH')
if not path:
    raise ValueError("DATA_PATH environment variable is not set")

filename = f"{path}\\Versuch2_Messungen\\versuch_2_graukeil.png"
src = cv2.imread(filename)
image = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

white = image[:, 0:202] # first pixels for white
light_grey = image[:, 223:395] # next pixels for light grey
mid_grey = image[:, 411:586] # next pixels for mid grey
dark_grey = image[:, 597:764] # next pixels for dark grey
black = image[:, 779:960] # last pixels for black

output_path = os.path.join(path, "Grauwertstufen") # output directory
os.makedirs(output_path, exist_ok=True) # create directory if it does not exist
# Save grayscale images
cv2.imwrite(os.path.join(output_path, "white.png"), white)
cv2.imwrite(os.path.join(output_path, "light_grey.png"), light_grey)
cv2.imwrite(os.path.join(output_path, "mid_grey.png"), mid_grey)
cv2.imwrite(os.path.join(output_path, "dark_grey.png"), dark_grey)
cv2.imwrite(os.path.join(output_path, "black.png"), black)

mean_white = np.mean(white)
mean_light_grey = np.mean(light_grey)
mean_mid_grey = np.mean(mid_grey)
mean_dark_grey = np.mean(dark_grey)
mean_black = np.mean(black)

std_white = np.std(white, ddof=1)
std_light_grey = np.std(light_grey, ddof=1)
std_mid_grey = np.std(mid_grey, ddof=1)
std_dark_grey = np.std(dark_grey, ddof=1)
std_black = np.std(black, ddof=1)

# Display mean and std as table
import pandas as pd
data = {
    'Grauwertstufe': ['Weiß', 'Hellgrau', 'Mittelgrau', 'Dunkelgrau', 'Schwarz'],
    'Mittelwert': [mean_white, mean_light_grey, mean_mid_grey, mean_dark_grey, mean_black],
    'Standardabweichung': [std_white, std_light_grey, std_mid_grey, std_dark_grey, std_black]
}
df = pd.DataFrame(data)
print(df.to_string(index=False))


window_name = 'Graukeil'
cv2.imshow(window_name, image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Grauwertstufe  Mittelwert  Standardabweichung
         Weiß  206.681321            3.276231
     Hellgrau  156.214400            2.834210
   Mittelgrau  108.265504            4.634426
   Dunkelgrau   61.489279            5.040507
      Schwarz   27.984126            2.667411


Routine, die die 10 Bilder einliest, in double umwandelt, die Farbbilder
in Grauwertbilder umrechnet und ihren pixelweisen Mittelwert berechnet. 
Das Mittelwertbild ist das Dunkelbild für die gewählte
Belichtungszeit. Das Dunkelbild wird kontrastmaximiert dargestellt.

In [3]:
images = []
# Save 10 darf frame images as double
for i in range(1, 11):
    filename = f"{path}\\Versuch2_Messungen\\versuch_2_dunkelbild{i}.png"
    src = cv2.imread(filename)
    image = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    image = image.astype(np.float64)
    images.append(image)

# Calculate mean for each pixel
mean_image = np.zeros(images[0].shape, dtype=np.float64)
for line in range(images[0].shape[0]):
    for row in range(images[0].shape[1]):
        summe = 0
        for image in images:
            summe += image[line][row]
        mean = summe / len(images)
        mean_image[line][row] = mean

# Convert mean image to uint8, BGR and save it
mean_image_uint8 = np.clip(mean_image, 0, 255).astype(np.uint8)
mean_image_uint8 = cv2.cvtColor(mean_image_uint8, cv2.COLOR_GRAY2BGR)
output_path_dunkelbild = os.path.join(path, "Dunkelbild") # output directory
os.makedirs(output_path_dunkelbild, exist_ok=True) # create directory if it does not exist
cv2.imwrite(os.path.join(output_path_dunkelbild, "dunkelbild_mittelwert.png"), mean_image_uint8)

# read the contrast maximized dunkelbild file 
image_path = os.path.join(output_path_dunkelbild, "dunkel_mittel_neu_kontrast_max.png") 
src = cv2.imread(image_path)
image = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
# show the dark frame image with maximized contrast
window_name = 'Dunkelbild Mittelwert Kontrastmaximiert'
cv2.imshow(window_name, image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Routine, die das Dunkelbild einliest, von einem Eingabebild subtrahiert und das korrigierte Bild abspeichert.

Erweiterung der Routine, dass
das Weißbild eingelesen, in double umgewandelt wird und so normiert wird, dass sein
Mittelwert 1 ist. Das durch Abzug des Dunkelbildes korrigierte Eingangsbild wird
anschliessend durch das normierte Weißbild dividiert.

In [7]:
# read the Dunkelbild file 
output_path_dunkelbild = os.path.join(path, "Dunkelbild") # output director
image_path = os.path.join(output_path_dunkelbild, "dunkelbild_mittelwert.png") 
src = cv2.imread(image_path)
dunkelbild = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

# read the Eingabebild file
filename = f"{path}\\Versuch2_Messungen\\versuch_2_graukeil.png"
src = cv2.imread(filename)
eingabebild = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

# subtract the Dunkelbild from the Eingabebild
result = cv2.subtract(eingabebild, dunkelbild)
result = cv2.cvtColor(result, cv2.COLOR_GRAY2BGR)
output_path_ergebnis = os.path.join(path, "Ergebnisbilder") # output directory
os.makedirs(output_path_ergebnis, exist_ok=True) # create directory if it does not exist
cv2.imwrite(os.path.join(output_path_ergebnis, "eingabebild-dunkelbild.png"), result)

# read the Weißbild file
output_path_weissbild = os.path.join(path, "Weissbild") # output director
image_path = os.path.join(output_path_weissbild, "weissbild_mittelwert.png") 
src = cv2.imread(image_path)
weissbild = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
weissbild = weissbild.astype(np.float64)
# Normierung des Weißbildes
weissbild = weissbild / np.mean(weissbild)
result = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY).astype(np.float64)
result = cv2.divide(result, weissbild)

cv2.imwrite(os.path.join(output_path_ergebnis, "eingabebild-dunkelbild_durch_weiss.png"), result)

Shape of result: (544, 960)
Shape of weissbild: (544, 960)


True

Routine, die 10 Weißbilder einliest und ihren Mittelwert berechnet.
Subtraktion des Dunkelbildes vom Mittelwertbild und Speicherung des resultierenden Weißbildes
Weißbild kontrastmaximiert darstellen.

In [ ]:
images = []
# Save 10 darf frame images as double
for i in range(1, 11):
    filename = f"{path}\\Versuch2_Messungen\\versuch_2_weissbild_{i}.png"
    src = cv2.imread(filename)
    image = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    image = image.astype(np.float64)
    images.append(image)

# Calculate mean for each pixel
mean_image = np.zeros(images[0].shape, dtype=np.float64)
for line in range(images[0].shape[0]):
    for row in range(images[0].shape[1]):
        summe = 0
        for image in images:
            summe += image[line][row]
        mean = summe / len(images)
        mean_image[line][row] = mean

# Convert mean image to uint8, BGR and save it
mean_image_uint8 = np.clip(mean_image, 0, 255).astype(np.uint8)
result = cv2.subtract(mean_image_uint8, dunkelbild)
result = cv2.cvtColor(result, cv2.COLOR_GRAY2BGR)
output_path_weissbild = os.path.join(path, "Weissbild") # output directory
os.makedirs(output_path_weissbild, exist_ok=True) # create directory if it does not exist
cv2.imwrite(os.path.join(output_path_weissbild, "weissbild_mittelwert.png"), result)

# read the contrast maximized weissbild file 
image_path = os.path.join(output_path_weissbild, "weiss_mittel_sub_dunkel_kontrast_max.png") 
src = cv2.imread(image_path)
image = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
# show the dark frame image with maximized contrast
window_name = 'Weissbild Mittelwert Kontrastmaximiert'
cv2.imshow(window_name, image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Überprüfung des Dunkelbildes auf dem Bildschirm auf stuck und hot pixels und
Weißbild auf dead pixels. 

Keine Stuck und Hot Pixel im Dunkelbild gefunden.
Keine Deadpixel im Weissbild gefunden.

Korrigiertes Graukeilbild einlesen und darstellen.

In [8]:
# read the corrected Graukeil file 
image_path = os.path.join(output_path_ergebnis, "eingabebild-dunkelbild_durch_weiss.png") 
src = cv2.imread(image_path)
image = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
# show the dark frame image with maximized contrast
window_name = 'Korrigierters Graukeilbild'
cv2.imshow(window_name, image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
filename = f"{path}\\Ergebnisbilder\\eingabebild-dunkelbild_durch_weiss.png"
src = cv2.imread(filename)
image = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

white = image[:, 0:202] # first pixels for white
light_grey = image[:, 223:395] # next pixels for light grey
mid_grey = image[:, 411:586] # next pixels for mid grey
dark_grey = image[:, 597:764] # next pixels for dark grey
black = image[:, 779:960] # last pixels for black

output_path = os.path.join(path, "Grauwertstufen_Ergebnisbild") # output directory
os.makedirs(output_path, exist_ok=True) # create directory if it does not exist
# Save grayscale images
cv2.imwrite(os.path.join(output_path, "white.png"), white)
cv2.imwrite(os.path.join(output_path, "light_grey.png"), light_grey)
cv2.imwrite(os.path.join(output_path, "mid_grey.png"), mid_grey)
cv2.imwrite(os.path.join(output_path, "dark_grey.png"), dark_grey)
cv2.imwrite(os.path.join(output_path, "black.png"), black)

mean_white = np.mean(white)
mean_light_grey = np.mean(light_grey)
mean_mid_grey = np.mean(mid_grey)
mean_dark_grey = np.mean(dark_grey)
mean_black = np.mean(black)

std_white = np.std(white, ddof=1)
std_light_grey = np.std(light_grey, ddof=1)
std_mid_grey = np.std(mid_grey, ddof=1)
std_dark_grey = np.std(dark_grey, ddof=1)
std_black = np.std(black, ddof=1)

# Display mean and std as table
import pandas as pd
corrected_result_data = {
    'Grauwertstufe': ['Weiß', 'Hellgrau', 'Mittelgrau', 'Dunkelgrau', 'Schwarz'],
    'Mittelwert': [mean_white, mean_light_grey, mean_mid_grey, mean_dark_grey, mean_black],
    'Standardabweichung': [std_white, std_light_grey, std_mid_grey, std_dark_grey, std_black]
}
df_result = pd.DataFrame(corrected_result_data)
print(df_result.to_string(index=False))

Grauwertstufe  Mittelwert  Standardabweichung
         Weiß  197.597390            2.393166
     Hellgrau  145.911626            1.680912
   Mittelgrau  104.985263            1.305853
   Dunkelgrau   61.866810            2.346287
      Schwarz   28.051064            2.097289


Ursprungsbild:
Grauwertstufe  Mittelwert  Standardabweichung
         Weiß  206.681321            3.276231
     Hellgrau  156.214400            2.834210
   Mittelgrau  108.265504            4.634426
   Dunkelgrau   61.489279            5.040507
      Schwarz   27.984126            2.667411

Differenz:
Grauwertstufe Mittelwert Standardabweichung
         Weiß   9.083931 0.883065
     Hellgrau  10.302774 1.153298
   Mittelgrau   3.280241 3.328573
   Dunkelgrau  -0.377531 2.694220
      Schwarz  -0.066938 0.570122

Fazit:
Standardabweichung wurde reduziert.
Jede Graustufe ist uniformer geworden.